## Case Study 1 : Suicide Rates
### 实例1: 自杀率
This case study with Pandas package uses a simple toy dataset from Kaggle. You can [download](https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016) here.

这个例子使用Kaggle上一个1985到2016年国家自杀率的数据集，可以从这里下载：[下载](https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016)

We read the csv and do some setup process below.

In [9]:
import pandas as pd
import numpy as np
import os

data_path = 'master.csv'

# Read the CSV file and rename columns
df = pd.read_csv(filepath_or_buffer=data_path).rename(columns={
    'suicides/100k pop': 'suicides_per_100k',
    ' gdp_for_year ($) ': 'gdp_year',
    'gdp_per_capita ($)': 'gdp_capita',
    'country-year': 'country_year'
})

# Remove commas and convert 'gdp_year' column to int64
df = df.assign(gdp_year=lambda _df: _df['gdp_year'].str.replace(',', '').astype(np.int64))

In [10]:
# We can see the column names below
df.columns

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides_per_100k', 'country_year', 'HDI for year', 'gdp_year',
       'gdp_capita', 'generation'],
      dtype='object')

By using unique, nunique and describe functions, you can access the overview of data quickly

Also, use head() and tail(), you can have a glimpse of the overall shape of data

In [11]:
# unique() will return unique elements in a column
print("Distinct Gender", df['sex'].unique())
print("Distinct Generations:", df['generation'].unique())

# nunique() will count the number of unique elements
print('Number of Distinct Countries:', df['country'].nunique())

# describe() will print common statistic data of this data frame
df.describe()

Distinct Gender ['male' 'female']
Distinct Generations: ['Generation X' 'Silent' 'G.I. Generation' 'Boomers' 'Millenials'
 'Generation Z']
Number of Distinct Countries: 101


,year,suicides_no,population,suicides_per_100k,HDI for year,gdp_year,gdp_capita
count,27820.000000,27820.000000,2.782000e+04,27820.000000,8364.000000,2.782000e+04,27820.000000
mean,2001.258375,242.574407,1.844794e+06,12.816097,0.776601,4.455810e+11,16866.464414
std,8.469055,902.047917,3.911779e+06,18.961511,0.093367,1.453610e+12,18887.576472
min,1985.000000,0.000000,2.780000e+02,0.000000,0.483000,4.691962e+07,251.000000
25%,1995.000000,3.000000,9.749850e+04,0.920000,0.713000,8.985353e+09,3447.000000
50%,2002.000000,25.000000,4.301500e+05,5.990000,0.779000,4.811469e+10,9372.000000
75%,2008.000000,131.000000,1.486143e+06,16.620000,0.855000,2.602024e+11,24874.000000
max,2016.000000,22338.000000,4.380521e+07,224.970000,0.944000,1.812071e+13,126352.000000


Below are some examples of usgae of indexing to select data from the data frame

In [12]:
# loc function - select certain rows
df.loc[2:5]

# iloc function
df.iloc[lambda x: x.index % 2 == 0] # select even number rows
df.iloc[1:3, :6] # select 1-3 rows and first 6 columns

,country,year,sex,age,suicides_no,population
1,Albania,1987,male,35-54 years,16,308000
2,Albania,1987,female,15-24 years,14,289700


Below are some examples of functions chain to analyze the dataset

In [13]:
# acquire unique generations in different age groups and change column name when printing
df.groupby('age').agg({'generation' : 'unique'}).rename(columns={'generation' : 'unique_generation'})

,unique_generation
age,
15-24 years,"[Generation X, Millenials]"
25-34 years,"[Boomers, Generation X, Millenials]"
35-54 years,"[Silent, Boomers, Generation X]"
5-14 years,"[Generation X, Millenials, Generation Z]"
55-74 years,"[G.I. Generation, Silent, Boomers]"
75+ years,"[G.I. Generation, Silent]"


In [17]:
# get countries and time with higher suicide rates
df.groupby(['country', 'year']).agg(suicides_sum=('suicides_per_100k',
                                                'sum')).sort_values('suicides_sum', ascending=False).head(10)
# Using nlargest is a more efficient way
df.groupby(['country', 'year']).agg(suicides_sum=('suicides_per_100k',
                                                'sum')).nlargest(10, columns='suicides_sum')


suicides_sum
country            year              
Lithuania          1995        639.30
                   1996        595.61
Hungary            1991        575.00
Lithuania          2000        571.80
Hungary            1992        570.26
Lithuania          2001        568.98
Russian Federation 1994        567.64
Lithuania          1998        566.36
                   1997        565.44
                   1999        561.53

In [18]:
# some pipe functions for future use
def log_head(df, head_count=10):
    print(df.head(head_count))
    return df

def log_columns(df):
    print(df.columns)
    return df

def log_shape(df):
    print(f'shape = {df.shape}')
    return df

In [20]:
# look for countries with high rate of suicide among female
df[df['sex'] == 'female'].pipe(log_shape).groupby(['country']).agg(
    sum_suicides_female=('suicides_per_100k', 'sum')).nlargest(10, columns='sum_suicides_female')


shape = (13910, 12)


,sum_suicides_female
country,
Republic of Korea,2755.13
Japan,2546.84
Singapore,2356.50
Hungary,2186.07
Austria,1971.65
Belgium,1937.60
Russian Federation,1879.37
France,1773.12
Suriname,1760.13


Extra: We can calculate the correlation between suicide and GDP

In [21]:
from sklearn.preprocessing import MinMaxScaler

def norm_df(df, columns):
    return df.assign(**{col: MinMaxScaler().fit_transform(df[[col]].values.astype(float))
                        for col in columns})

for sex in ['male', 'female']:
    print(sex)
    print(
        df.query(f'sex == "{sex}"')
        .groupby(['country'])
        .agg(
            suicides_per_100k_sum=('suicides_per_100k', 'sum'),
            gdp_year_mean=('gdp_year', 'mean'))
        .pipe(norm_df, columns=['suicides_per_100k_sum', 'gdp_year_mean'])
        .corr(method='spearman')        
    )
    print('\n')

male
                       suicides_per_100k_sum  gdp_year_mean
suicides_per_100k_sum               1.000000       0.421218
gdp_year_mean                       0.421218       1.000000


female
                       suicides_per_100k_sum  gdp_year_mean
suicides_per_100k_sum               1.000000       0.452343
gdp_year_mean                       0.452343       1.000000


